# Lab 1: Build a Similarity Search for YouTube Transcripts

In this notebook, you will use the AzureOpenAI client to get the text embeddings of a string and perform a cosine similarity comparison against the transcripts from [Boston Azure Youtube channel](https://www.youtube.com/bostonazure) to find the videos with the highest similarity.

## Learning Objectives

* Load the variables in the .env file
* Connect to AzureOpenAI in python
* Load the transcript file and create a pandas data frame
* Calculate the similarity of a transcript's embeddings to the text embedding
* Output the most similar videos with a url formatted to navigate to the 5 min section that was found most similiar


## Similarity

TODO: Description of what similarity is and some references to learn more

### Step 1: load environment variables and create the AzureOpenAI client


In [1]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01",
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
  )

model = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")

### Step 2: Set the threshold for the similarity score we want to use and version of the transcript file.    

In [2]:
SIMILARITIES_RESULTS_THRESHOLD = 0.70
DATASET_NAME = "./prep/output/master_enriched.json"

### Step 3: Create some utility methods

In [9]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    print(f"")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"")

### Step 4: Load the transcript file (and take a look at what is in it)

In [12]:
pd_vectors = load_dataset(DATASET_NAME)

Notice that this version of the transcripts file has the embeddings in it already for the 5 minute chunks of the transcript text. The embeddings are the ada_v2 column.

In [ ]:
pd_vectors

### Step 4: Try it out

I've put some default text in for a good example, but you should change the query to your own search and see what comes back.

In [ ]:
query = "What is langchain?"

videos = get_videos(query, pd_vectors, 5)
display_results(videos, query)



### Reference
This code is a modified version of this notebook: [ai-beginners-embeddings](https://github.com/gloveboxes/ai-beginners-embeddings/blob/main/main.ipynb)